In [44]:


import requests
from bs4 import BeautifulSoup
import csv
import os
import itertools

#Revisant robots.txt es veu sobre quines pagines no es pot fer scrapping. 
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('http://www.amazon.com/robots.txt')
rp.read()
url = 'https://www.amazon.es/gp/bestsellers/books?pf_rd_r=3SGMV0M1FJ0N0CB7MQC0&pf_rd_p=d3259a28-7b77-56ff-ade0-8a02db82407a&pd_rd_r=0c54ad8c-b4d5-4d1a-a3bb-3246e048bdc2&pd_rd_w=m1O3G&pd_rd_wg=ES5Zj&ref_=pd_gw_ri'
user_agent = 'wswp'
print(rp.can_fetch(user_agent, url))

#Es declara la variable on s'enmagatzemaran els resultats

llista_llibres = []

#Es crea l'scrapp
#El llistat de llibres de Amazon es divideix en dues pagines, pel que hem de llegir aquestes dues.
for pagina in range(1,3):
    url = "https://www.amazon.es/gp/bestsellers/books/ref=zg_bs_pg_1/259-0380097-5900722?ie=UTF8&pg=%d" % pagina
    page = requests.get(url)
    soup = BeautifulSoup(page.content)

    # Guardem en la variable llistat el contingut de tots els llibres
    llistat = soup.find(class_="a-ordered-list a-vertical")

    # Recorrem llibre a llibre (aok-inline-block zg-item) buscant les característiques de cada un d'ells. S'Afegeix 
    #la opció NULL en cas de que la característica no estigui documentada.

    for descripcio in llistat.find_all('span', class_="aok-inline-block zg-item"):
    
        nom = (descripcio.find(class_="p13n-sc-truncate").string).strip()
        try:
            autor = descripcio.find(class_="a-row a-size-small").string
        except:
            autor = "NULL"
        try:
            puntuacio = descripcio.find(class_="a-icon-alt").string
        except:
            puntacio = "NULL"
        try:
            num_resenyes = descripcio.find(class_="a-size-small a-link-normal").string
        except: 
            num_resenyes = "NULL"
        try:
            categoria = descripcio.find(class_="a-size-small a-color-secondary").string
        except:
            categoria = "NULL"
        preu = descripcio.find(class_="p13n-sc-price").string
    
        # Copiem les característiques de cada llibre en una línea del CSV final, utilitzant ";" com a separador
        linea = nom + ";" + autor + ";" + puntuacio + ";" + num_resenyes + ";" + categoria + ";" + preu
        llista_llibres.append(linea)

# Escrivim un fitxer csv amb les dades dels llibres més venuts d'amazon
with open('llibres.csv', 'w', newline='') as csvFile:
    csvFile.write("Nom;Autor;Puntuacio;Num_resenyes;Categoria;Preu\n")
    for llibre in llista_llibres:
        csvFile.write(llibre)
        csvFile.write('\n')



True
